Camel 3 Function 
=====================


In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
# include("../../../src/lib_func.jl") # Include the library of functions

# Constants and Parameters
d = 2 # Initial Degree 
const n, a, b = 2, 5, 1 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .5 , 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm

f = camel # Objective function

  Activating project at `~/Globtim.jl`


camel (generic function with 1 method)

In [2]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        println("Current degree: ", d)
        global d += 1
    end
end;

current L2-norm: 88.06221424717013
Number of samples: 8
Current degree: 2
current L2-norm: 56.53376717102288
Number of samples: 12
Current degree: 3
current L2-norm: 7.168172657465275
Number of samples: 17
Current degree: 4
current L2-norm: 5.469105102636791
Number of samples: 22
Current degree: 5
attained the desired L2-norm: 1.4139296557579468e-13


In [6]:
using DynamicPolynomials, DataFrames
using HomotopyContinuation, ProgressLogging

@polyvar(x[1:n]) # Define polynomial ring 
pol = main_nd(x, n, d, poly_approx.coeffs) 

grad = differentiate.(pol, x)
sys = System(grad)
println("The system is of degree: ", d-1)

The system is of degree: 5


In [4]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false);

Computing mixed cells... 2    Time: 0:00:00
Computing mixed cells... 6    Time: 0:00:00
  mixed_volume:  25


Tracking 25 paths...   8%|██▍                           |  ETA: 0:02:10



Tracking 25 paths... 100%|██████████████████████████████| Time: 0:00:11
  # paths tracked:                  25
  # non-singular solutions (real):  25 (15)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         25 (15)


In [7]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

In [8]:
using GLMakie
# Extract coordinates and function values
coords = poly_approx.scale_factor * poly_approx.grid
z_coords = poly_approx.z;

In [9]:
if size(coords)[2] == 2  # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Camel Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with a darker core
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=:black, label="Sampled Data Core")

    # Plot the sampled data with adjusted size and color to create a halo effect
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=4, color=z_coords, colormap=:viridis, label="Sampled Data Halo")

    # Plot the exact approximant critical points with adjusted size
    scatter!(ax, df.x, df.y, df.z, markersize=10, color=:red, label="Exact approximant critical points")

    display(fig)
end

GLMakie.Screen(...)